In [9]:
import pandas as pd

filename = 'training_datasets/training_dataset.csv'


features = ['Time Stamp', 'T', 'Po', 'P', 'Pa', 'U', 'DD', 'Ff', 'ff10', 'ff3', 'N', 'WW', 'W', 'Tn', 'Tx', 'Cl', 'Nh', 'H', 'Cm', 'Ch', 'VV', 'Td', 'tR', 'RRR']# 文档中要求的特征

# 保留文档中要求的特征
df = pd.read_csv(filename, delimiter=',', encoding='gb18030', low_memory=False)
df = df[features]

output_filename = 'training_datasets/training_dataset_0.csv'# 原始数据集
df.to_csv(output_filename)

print(df.isnull().sum())

Time Stamp        0
T                 3
Po              242
P               235
Pa            13735
U                 5
DD                0
Ff                0
ff10          40774
ff3           32882
N              8391
WW                0
W             26432
Tn            26703
Tx            27371
Cl            33023
Nh            20900
H             11298
Cm            32665
Ch            34002
VV             1115
Td                2
tR            26710
RRR           26710
dtype: int64


In [13]:
df1 = df
df1['RRR'] = pd.to_numeric(df1['RRR'], errors='coerce')
rrr_values = df1['RRR'].loc[~df1['RRR'].isnull()].to_numpy()
print(rrr_values.min())

0.1


In [2]:
# 确定的处理
df.drop(['ff10'], axis=1, inplace=True)# 没有数据，直接删除特征
df.dropna(subset=['T', 'Po', 'P', 'U', 'VV', 'Td'], inplace=True)# 少量缺失，直接删除对应数据

# 不确定的处理
df.drop(['ff3', 'Cl', 'Cm', 'Ch'], axis=1, inplace=True)# 数据缺失过多，考虑删除特征

df.reset_index(drop=True, inplace=True)
print(df.isnull().sum())

Time Stamp        0
T                 0
Po                0
P                 0
Pa            13473
U                 0
DD                0
Ff                0
N              8389
WW                0
W             25576
Tn            25417
Tx            26217
Nh            20082
H             10853
VV                0
Td                0
tR            25506
RRR           25506
dtype: int64


In [3]:
# 补"0"
df['N'].fillna('无云', inplace=True)
df['WW'].fillna('云量发展情况没有进行观测或无法观测。 ', inplace=True)
df['Nh'].fillna('0', inplace=True)
df['H'].fillna('2500或更高，或无云。', inplace=True)
df['RRR'].fillna('无降水', inplace=True)
df['tR'].fillna('0', inplace=True)


# 线性插值
df['Pa'].interpolate(method='linear', inplace=True)
df['Tn'].interpolate(method='linear', inplace=True)
df['Tx'].interpolate(method='linear', inplace=True)

print(df.isnull().sum())
df_complete = df.dropna()
print(df_complete.shape[0])

output_filename_1 = 'training_datasets/training_dataset_1.csv'
df.to_csv(output_filename_1)


Time Stamp        0
T                 0
Po                0
P                 0
Pa                0
U                 0
DD                0
Ff                0
N                 0
WW                0
W             25576
Tn                0
Tx                0
Nh                0
H                 0
VV                0
Td                0
tR                0
RRR               0
dtype: int64
13835


In [7]:
df1 = df.drop(['Time Stamp', 'DD', 'WW', 'W', 'H', 'N', 'Nh'], axis=1, inplace=False)
df['VV'].replace('低于 0.1', 0, inplace=True)
df1['RRR'].replace('无降水', -1, inplace=True)
df1['RRR'].replace('降水迹象', 0, inplace=True)

In [8]:
corr = df1.corr(method='pearson', numeric_only=False)['RRR']
print(corr)

T      0.049321
Po    -0.150688
P     -0.151695
Pa     0.028532
U      0.245651
Ff     0.059737
Tn     0.099555
Tx     0.056244
VV    -0.083120
Td     0.141937
tR     0.252274
RRR    1.000000
Name: RRR, dtype: float64
